https://www.youtube.com/watch?v=V1-Hm2rNkik

#pandas를 이용하여 불러오기

In [ ]:
import pandas as pd
df = messages = pd.read_csv('~~', sep='\t',
                            names = ['~~','~~'])


x,y에 대한 전처리

영상에서 y=spam으로 처리
- 따라서 one hot encoding을 통해 숫자로 처리

In [ ]:
y=list(pd.get_dummies(y, drop_first=True)['~~'])

#sklearn을 이용한 학습 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.20, random_state=42)

#transformers의 tokenizer를 이용한 embedding

##process

###1. Call the pretrained model(Distilbert)




In [ ]:
! pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFase.from_pretrained('distilbert-base-uncased')

###2. Call the tokenizer(Distilbert)

In [ ]:
train_encodings = tokenizer(x_train, truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

train_encodings의 결과는 numerical value에 padding이 덧씌워진 형태

In [ ]:
train_encodings

###3. Convert these encodings into Dataset object

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_slices((
    dict(test_encodings),
    y_test
))


train_dataset의 결과는 shapes, attention_mask, types등으로 반환

In [ ]:
train_dataset

In [ ]:
 from transformers import TFTrainingArguments, TFDistilbertForSequenceClassification, TFTrainer

 training_args = TFTrainingArguments(
    output_dir = './results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
 )

In [ ]:
with training_args.strategy.scope() :
  model = TFDistilbertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

trainer.train()



In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
# check output
trainer.predict(test_dataset)

In [ ]:
 trainer.predict(test_dataset)[1].shape

In [ ]:
output = trainer.predict(test_dataset)[1]

In [ ]:
# confusion matrix of result
from sklearn.metrics import confusion_matrix

cm = confusino_matrix(y_test, output)
cm

In [ ]:
trainer.save_model('senti_model')